# Data Wrangling:

In [2]:
#import relevent packages
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#set api_key var
api_key = ################

In [5]:
#get n house bills

#create empty target data frame
bills113 = pd.DataFrame()

#create indices for random sampling of data
offset_indices = np.arange(347)
np.random.shuffle(offset_indices)

n = 0

while n < 100:
    
    #pop the offset amount
    offset, offset_indices = 20*offset_indices[-1], offset_indices[:-1]
    
    #query database and convert to DataFrame
    response = requests.get('https://api.propublica.org/congress/v1/113/house/bills/introduced.json', 
                            {'offset': offset}, headers = {'X-API-Key': api_key})
    
    response_body = response.json()
    bills = response_body['results'][0]['bills']
    df = pd.DataFrame(bills)
    
    #filter out resolutions and ammendments
    df = df[df.bill_type == 'hr'] 
    
    #count new data points
    n += df.shape[0]
    
    #append new data
    bills113 = pd.concat([bills113, df])
    
print(bills113.shape)

(112, 33)


In [13]:
#filter relevant columns
keep_cols = ['bill_id', 'bill_slug', 'title', 'short_title', 'sponsor_title', 'sponsor_id',
             'sponsor_name', 'sponsor_party', 'introduced_date', 'active', 'last_vote', 'house_passage',
            'senate_passage', 'enacted', 'vetoed', 'cosponsors', 'cosponsors_by_party', 'committees',
            'committee_codes', 'subcommittee_codes', 'primary_subject', 'summary', 'summary_short', 
            'latest_major_action_date', 'latest_major_action']
bills113 = bills113[keep_cols]
bills113.head()

,bill_id,bill_slug,title,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_party,introduced_date,active,...,cosponsors,cosponsors_by_party,committees,committee_codes,subcommittee_codes,primary_subject,summary,summary_short,latest_major_action_date,latest_major_action
0,hr5604-113,hr5604,To direct the Secretary of the Army to revise ...,To direct the Secretary of the Army to revise ...,Rep.,H000067,Ralph M. Hall,R,2014-09-18,False,...,0,{},House Transportation and Infrastructure Committee,[HSPW],[HSPW02],Water Resources Development,"Directs the Secretary of the Army, acting thr...","Directs the Secretary of the Army, acting thr...",2014-09-19,Referred to the Subcommittee on Water Resource...
1,hr5603-113,hr5603,To provide for the conveyance of the Lake Fann...,To provide for the conveyance of the Lake Fann...,Rep.,H000067,Ralph M. Hall,R,2014-09-18,False,...,0,{},House Agriculture Committee,[HSAG],[HSAG15],Public Lands and Natural Resources,Directs the Secretary of Agriculture (USDA) to...,Directs the Secretary of Agriculture (USDA) to...,2014-10-23,"Referred to the Subcommittee on Conservation, ..."
2,hr5602-113,hr5602,To designate the facility of the United States...,To designate the facility of the United States...,Rep.,H001063,Janice Hahn,D,2014-09-18,False,...,0,{},House Oversight and Government Reform Committee,[HSGO],[],Government Operations and Politics,Designates the facility of the United States P...,Designates the facility of the United States P...,2014-09-19,Sponsor introductory remarks on measure. (CR E...
3,hr5601-113,hr5601,To provide amounts from the recent settlement ...,Foreclosure Restitution Act of 2014,Rep.,G000556,Alan Grayson,D,2014-09-18,False,...,0,{},House Financial Services Committee,[HSBA],[],Housing and Community Development,Foreclosure Restitution Act of 2014 - Appropri...,Foreclosure Restitution Act of 2014 - Appropri...,2014-09-18,Referred to the House Committee on Financial S...
4,hr5600-113,hr5600,To amend the Small Business Act to establish r...,Franchise Financing Improvement Act of 2014,Rep.,G000546,Sam Graves,R,2014-09-18,False,...,0,{},House Small Business Committee,[HSSM],[],Commerce,Franchise Financing Improvement Act of 2014 - ...,Franchise Financing Improvement Act of 2014 - ...,2014-09-18,Referred to the House Committee on Small Busin...


In [14]:
#get members

#query database and convert to DataFrame
response = requests.get("https://api.propublica.org/congress/v1/113/house/members.json",
                       headers = {'X-API-Key': api_key})
response_body = response.json()
members = response_body['results'][0]['members']
df = pd.DataFrame(members)

#filter for relevant columns
keep_cols = ['id', 'gender', 'leadership_role', 'seniority', 'missed_votes_pct',
             'votes_with_party_pct', 'votes_against_party_pct' ]
members113 = df[keep_cols]


In [15]:
members113.head()

,id,gender,leadership_role,seniority,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,A000370,F,None,2,4.17,97.83,2.17
1,A000055,M,None,18,9.14,94.40,5.50
2,A000361,M,None,12,2.45,92.86,6.93
3,A000367,M,None,4,0.00,77.31,22.60
4,A000369,M,None,4,8.14,94.47,5.44


In [16]:
#merge frames
data113 = bills113.merge(members113, how= 'left', left_on = 'sponsor_id', right_on = 'id')
data113.head()

,bill_id,bill_slug,title,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_party,introduced_date,active,...,summary_short,latest_major_action_date,latest_major_action,id,gender,leadership_role,seniority,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,hr5604-113,hr5604,To direct the Secretary of the Army to revise ...,To direct the Secretary of the Army to revise ...,Rep.,H000067,Ralph M. Hall,R,2014-09-18,False,...,"Directs the Secretary of the Army, acting thr...",2014-09-19,Referred to the Subcommittee on Water Resource...,H000067,M,None,34,7.39,94.32,5.59
1,hr5603-113,hr5603,To provide for the conveyance of the Lake Fann...,To provide for the conveyance of the Lake Fann...,Rep.,H000067,Ralph M. Hall,R,2014-09-18,False,...,Directs the Secretary of Agriculture (USDA) to...,2014-10-23,"Referred to the Subcommittee on Conservation, ...",H000067,M,None,34,7.39,94.32,5.59
2,hr5602-113,hr5602,To designate the facility of the United States...,To designate the facility of the United States...,Rep.,H001063,Janice Hahn,D,2014-09-18,False,...,Designates the facility of the United States P...,2014-09-19,Sponsor introductory remarks on measure. (CR E...,H001063,F,None,4,0.58,95.62,4.30
3,hr5601-113,hr5601,To provide amounts from the recent settlement ...,Foreclosure Restitution Act of 2014,Rep.,G000556,Alan Grayson,D,2014-09-18,False,...,Foreclosure Restitution Act of 2014 - Appropri...,2014-09-18,Referred to the House Committee on Financial S...,G000556,M,None,2,2.91,94.14,5.78
4,hr5600-113,hr5600,To amend the Small Business Act to establish r...,Franchise Financing Improvement Act of 2014,Rep.,G000546,Sam Graves,R,2014-09-18,False,...,Franchise Financing Improvement Act of 2014 - ...,2014-09-18,Referred to the House Committee on Small Busin...,G000546,M,None,14,6.23,93.77,6.14


In [ ]:
#possible expansion, get sponsor committee data